In [21]:
import pandas as pd
import numpy as np
import math
import random
pd.set_option('display.max_rows', 500)

In [26]:
master_data = pd.read_csv("../../Data/filtered/master-data.csv", sep=",", encoding='Latin-1', low_memory=False)

In [27]:
master_data.head()

,compensation,year,country,age,gender,IT_experience_in_years,company_size,occupation,proficient_languages,desktop_OS,job_satisfaction
0,Student / Unemployed,2011,Africa,below_18,female,<2,Start Up (1-25),Web Application Developer,['JavaScript'],Linux,FML
1,NaN,2011,Other Europe,25_to_34,male,41310,Mature Small Business (25-100),Server Programmer,"['SQL', 'C']",Windows 7,So happy it hurts
2,NaN,2011,India,25_to_34,female,41435,Mid Sized (100-999),Server Programmer,"['JavaScript', 'SQL']",Linux,NaN
3,Student / Unemployed,2011,Germany,below_18,female,41310,Student,Student,['Haskell'],Linux,I enjoy going to work
4,"$80,000 - $100,000",2011,Other Asia,35_to_44,male,11,Start Up (1-25),"Executive (VP of Eng, CTO, CIO, etc.)","['JavaScript', 'CSS', 'PHP', 'SQL', 'C++', 'C'...",Linux,It pays the bills


In [28]:
#begin replacing values 
master_data['company_size'] = master_data['company_size'].replace(['Start Up (1-25)',
 'Mature Small Business (25-100)',
 'Mid Sized (100-999)',
 'Student',
 'Fortune 1000 (1,000+)',
 'nan',
 'Other (not working, consultant, etc.)',
 '101-999',
 '26-100',
 '1,000-3,000',
 '1/25/2013',
 '3001',
 '100',
 '1/5/2014',
 '50-100',
 '6/15/2014',
 '16-30',
 '31-50',
 'unknown',
 '100-499 employees',
 'I am not part of a company',
 '10-19 employees',
 '5-9 employees',
 '20-99 employees',
 '1-4 employees',
 '500-999 employees',
 '1,000-4,999 employees',
 '10,000+ employees',
 '5,000-9,999 employees',
 'I am not sure',
 'Other (please specify)',
 '20 to 99 employees',
 '10,000 or more employees',
 '10 to 19 employees',
 'Fewer than 10 employees',
 '5,000 to 9,999 employees',
 '100 to 499 employees',
 '1,000 to 4,999 employees',
 '500 to 999 employees',
 "I don't know",
 'I prefer not to answer',
 'Just me - I am a freelancer, sole proprietor, etc.',
 '2-9 employees',
 '2 to 9 employees',
 'I donÃ\x83Â¢Ã\x82Â\x80Ã\x82Â\x99t know', 'nan'],
    ['< 100',
'< 100',
'500 - 999',
'Student',
'1,000 - 4,999',
'nan',
'Other',
'500 - 999',
'< 100',
'1,000 - 4,999',
'nan',
'1,000 - 4,999',
'100 - 499',
'nan',
'< 100',
'nan',
'< 100',
'< 100',
'Unknown',
'100 - 499',
'Other',
'< 100',
'< 100',
'< 100',
'< 100',
'500 - 999',
'1,000 - 4,999',
'10,000 +',
'5,000 - 9,999',
'Unknown',
'Other',
'< 100',
'10,000 +',
'< 100',
'< 100',
'5,000 - 9,999',
'100 - 499',
'1,000 - 4,999',
'500 - 999',
'Unknown',
'Unknown',
'Other',
'Unknown',
'< 100',
'Unknown', 'nan'])


In [29]:
#preview the changes made for company size
master_data.head()
#preview unique list of grouping for company size
list(master_data["company_size"].unique())

['< 100',
 '500 - 999',
 'Student',
 '1,000 - 4,999',
 'Other',
 nan,
 'nan',
 '100 - 499',
 'Unknown',
 '10,000 +',
 '5,000 - 9,999',
 'I donÃ\x83Â\x83Ã\x82Â\x83Ã\x83Â\x82Ã\x82Â¢Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â\x80Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â\x99t know']

In [38]:
master_data.groupby(["year", "company_size"])["company_size"].count()

year  company_size
2011  1000_to_4999      543
      500_to_999        447
      Student           203
      Unknown           222
      below_100        1314
2012  1000_to_4999     1007
      500_to_999       1049
      Student           505
      Unknown           508
      below_100        2895
2013  1000_to_4999     1739
      500_to_999       1467
      Unknown          4750
      below_100        1478
2014  100_to_499       1647
      Unknown          4184
      below_100        1515
2015  Unknown         25492
2016  1000_to_4999     3579
      100_to_499       6771
      5000_to_9999     1363
      500_to_999       2295
      Unknown         17384
      above_10000      4680
      below_100       19264
2017  1000_to_4999     3811
      100_to_499       7254
      5000_to_9999     1598
      500_to_999       2472
      Unknown         13924
      above_10000      5654
      below_100       16432
2018  1000_to_4999     5534
      100_to_499       9931
      5000_to_9999     2188
 

In [35]:
#one more clean up step for company size, notice the last 500-999 does not have the spaces
def normalise_company_size(row):
    company_size = row 
    if row in ["< 100"]:
        company_size= "below_100"
    elif row in ["100 - 499"]:
        company_size= "100_to_499"
    elif row in ["500 - 999"]:
        company_size= "500_to_999"
    elif row in ["1,000 - 4,999"]:
        company_size= "1000_to_4999"
    elif row in ["5,000 - 9,999"]:
        company_size= "5000_to_9999"
    elif row in ["10,000 +"]:
        company_size="above_10000"
    elif ("I don" in row) or (row in ["Other", "nan"]):
        company_size= "Unknown"
    return company_size

In [36]:
master_data["company_size"] = master_data["company_size"].fillna("Unknown")
master_data["company_size"] = master_data["company_size"].apply(normalise_company_size)

In [41]:
# Dropping the records by students 
master_data = master_data[master_data.company_size != 'Student']

In [43]:
# Final list
master_data["company_size"].unique().tolist()

['below_100',
 '500_to_999',
 '1000_to_4999',
 'Unknown',
 '100_to_499',
 'above_10000',
 '5000_to_9999']

In [44]:
# Write the normalised company size to master data

(
    master_data.to_csv("../../Data/filtered/master-data.csv", index=False)
)
